In [240]:
import pandas as pd
import numpy as np

In [241]:
df=pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [242]:
df = df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [243]:
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [244]:
# train test split
from sklearn.model_selection import train_test_split
x = df.drop(['Survived'],axis=1)
y = df[['Survived']]

In [245]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

In [246]:
x_train.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [247]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

# 1 impute missing vlaues -> Age,Embarked
# 2 one hot encoding -> Sex,Embarked
# 3 Scaling -> Min-Max-Scaler becoz we perform feature selection
# 4 Feature Selection
# 5 Model training

# columns : Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')
#                       0      1      2      3       4         5          6      


# after encoding
# cols = 

In [248]:
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [249]:
trf2 = ColumnTransformer([
    ('encode_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,3])
],remainder='passthrough')

In [250]:
trf3 = ColumnTransformer([
    ('scaling_all_cols',MinMaxScaler(),slice(-1))
],remainder='passthrough')

In [251]:
from sklearn.feature_selection import SelectKBest,chi2
trf4 = SelectKBest(score_func=chi2,k=8)

In [252]:
from sklearn.tree import DecisionTreeClassifier

trf5 = DecisionTreeClassifier()

### Create Pipeline

In [253]:
from sklearn.pipeline import Pipeline , make_pipeline
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [254]:
# Alternate syntax 
# make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [258]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encode_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 3])])),
                ('trf3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaling_all_cols',
                                                  MinMaxScaler(),
                                                  slice(None, -1, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7566af071300>)),
                ('trf5', DecisionTreeClassifier())])

In [259]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('encode_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 3])]),
 'trf3': ColumnTransformer(remainder='passthrough',
                   transformers=[('scaling_all_cols', MinMaxScaler(),
                                  slice(None, -1, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x7566af071300>),
 'trf5': DecisionTreeClassifier()}

In [264]:
pipe.named_steps['trf1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [266]:
pipe.named_steps['trf1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder',
  FunctionTransformer(accept_sparse=True, check_inverse=False,
                      feature_names_out='one-to-one'),
  [0, 1, 3, 4, 5])]

In [269]:
from sklearn.metrics import accuracy_score
y_pred = pipe.predict(x_test)
print("Model Accuracy : ",accuracy_score(y_test,y_pred))

Model Accuracy :  0.7686567164179104


### Cross Validation Score Using PipeLine

In [272]:
from sklearn.model_selection import cross_val_score

print("Model Cross validation score : ",cross_val_score(pipe,x_train,y_train,cv=5,scoring='accuracy').mean())

Model Cross validation score :  0.8026580645161291


### GridSearchin Using PipeLine

In [273]:
# TODO : Not a topic , understood it later

# gridsearchcv
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [275]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('encode_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          3])])),
                                       ('trf3',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scaling_all_cols',
                                                                         MinMaxScaler(),
                                                                         slice(None, -1, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7566af071300>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [276]:
grid.best_score_

0.7994451612903226

In [277]:
grid.best_params_

{'trf5__max_depth': None}

### Exporting the PipeLine

In [278]:
import pickle
pickle.dump(pipe,open('models/pipe.pkl','wb'))